## Imports

In [4]:
import torch
import torch.utils.data as Data
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torchvision import transforms as T, models
from PIL import Image
from pathlib import Path
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from transformers import get_cosine_schedule_with_warmup
from sklearn.metrics import accuracy_score, confusion_matrix
from torchsummary import summary

import warnings
warnings.filterwarnings("ignore")


In [5]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [6]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
    torch.cuda.empty_cache()
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cuda


## Reading/Processing the Data

In [7]:
IMAGE_PATH = Path('./plant-pathology-2020-fgvc7/images')

def image_path(file_stem):
    return IMAGE_PATH/f'{file_stem}.jpg'

In [8]:
train_df = pd.read_csv('./plant-pathology-2020-fgvc7/train.csv')
test_df = pd.read_csv('./plant-pathology-2020-fgvc7/test.csv')

train_paths = train_df['img_file'] = train_df['image_id'].apply(image_path)
test_paths = test_df['img_file'] = test_df['image_id'].apply(image_path)

train_labels = train_df[['healthy','multiple_diseases','rust','scab']]

In [9]:
train_paths, valid_paths, train_labels, valid_labels = train_test_split(
    train_paths, train_labels, test_size = 0.2, random_state=23, stratify = train_labels)
train_paths.reset_index(drop=True,inplace=True)
train_labels.reset_index(drop=True,inplace=True)
valid_paths.reset_index(drop=True,inplace=True)
valid_labels.reset_index(drop=True,inplace=True)

### Creating a custom dataset object

In [10]:
class LeafDataset(Data.Dataset):
    def __init__(self, img_paths, labels=None, train=True, test=False):
        self.img_paths = img_paths
        self.train = train
        self.test = test
        
        if not self.test:
            self.labels = labels
        
        self.train_transform = T.Compose([T.RandomRotation(25),
                                          T.RandomHorizontalFlip(),
                                          T.RandomVerticalFlip(),])
        self.test_transform = T.Compose([T.CenterCrop(256),
                                         T.RandomRotation(25),
                                         T.RandomHorizontalFlip(),
                                         T.RandomVerticalFlip(),])
        self.default_transform = T.Compose([T.ToTensor(),
                                            T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),]) # ImageNet Stats
        
    def __len__(self):
        return self.img_paths.shape[0]
    
    
    def __getitem__(self, i):
        image = Image.open(self.img_paths[i]).resize((512, 512))
        if not self.test:
            label = torch.tensor(np.argmax(self.labels.loc[i, :].values))
        if self.train:
            image = self.train_transform(image)
        elif self.test:
            image = self.test_transform(image)
        image  = self.default_transform(image)
        
        
        return image, label if not self.test else image

### Define training, validataion and testing functions

In [32]:
def training(model, data_loader, optim, scheduler, loss_fn, acc_fn):
    running_loss = 0
    preds_for_acc = []
    labels_for_acc = []
    
    pbar = tqdm(total=len(data_loader), desc='Training')
    
    for idx, (images, labels) in enumerate(data_loader):
        images, labels = images.to(device), labels.to(device)
        model.train()
        optim.zero_grad()
        scores = model(images)
        loss = loss_fn(scores, labels)
        loss.backward()
        optim.step()
        scheduler.step()
        
        running_loss += loss.item() * labels.shape[0]
        labels_for_acc = np.concatenate((labels_for_acc, labels.cpu().numpy()), 0)
        preds_for_acc = np.concatenate(
            (preds_for_acc, np.argmax(scores.cpu().detach().numpy(), 1)), 0)
        
        pbar.update()
    
    pbar.close()
    
    return running_loss / TRAIN_SIZE, acc_fn(labels_for_acc, preds_for_acc)


def validation(model, data_loader, loss_fn, acc_fn, confusion_matrix):
     
    running_loss = 0
    preds_for_acc = []
    labels_for_acc = []
    
    pbar = tqdm(total = len(data_loader), desc='Validation')
    
    with torch.no_grad():       #torch.no_grad() prevents Autograd engine from storing intermediate values, saving memory
        for idx, (images, labels) in enumerate(data_loader):
            images, labels = images.to(device), labels.to(device)
            model.eval()
            scores = model(images)
            loss = loss_fn(scores, labels)
            
            running_loss += loss.item() * labels.shape[0]
            labels_for_acc = np.concatenate((labels_for_acc, labels.cpu().numpy()), 0)
            preds_for_acc = np.concatenate((preds_for_acc, np.argmax(scores.cpu().detach().numpy(), 1)), 0)
            
            pbar.update()
            
        accuracy = acc_fn(labels_for_acc, preds_for_acc)
        conf_mat = confusion_matrix(labels_for_acc, preds_for_acc)
    
    pbar.close()
    return running_loss/VALID_SIZE, accuracy, conf_mat


def testing(model, data_loader):
    
    preds_for_output = np.zeros((1,4))
    
    with torch.no_grad():
        pbar = tqdm(total = len(loader))
        for _, images in enumerate(loader):
            images = images.to(device)
            model.eval()
            scores = net(images)
            preds_for_output = np.concatenate((preds_for_output, scores.cpu().detach().numpy()), 0)
            pbar.update()
    
    pbar.close()
    return preds_for_output

### Initialization

In [47]:
BATCH_SIZE = 16
NUM_EPOCHS = 30
TRAIN_SIZE = train_labels.shape[0]
VALID_SIZE = valid_labels.shape[0]

In [48]:
train_dataset = LeafDataset(train_paths, train_labels)
trainloader = Data.DataLoader(train_dataset, shuffle=True, batch_size = BATCH_SIZE, num_workers = 2)

valid_dataset = LeafDataset(valid_paths, valid_labels, train = False)
validloader = Data.DataLoader(valid_dataset, shuffle=False, batch_size = BATCH_SIZE, num_workers = 2)

test_dataset = LeafDataset(test_paths, train = False, test = True)
testloader = Data.DataLoader(test_dataset, shuffle=False, batch_size = BATCH_SIZE, num_workers = 2)

### DenseNet

In [49]:
densenet = models.densenet161(pretrained=True)

In [50]:
num_filters = densenet.classifier.in_features
densenet.classifier = nn.Sequential(
    nn.Linear(num_filters, train_labels.shape[1])
)

In [51]:
optimizer = torch.optim.Adam(densenet.parameters(), lr=8e-4, weight_decay = 1e-3)
num_train_steps = int(len(train_dataset) / BATCH_SIZE * NUM_EPOCHS)
scheduler = get_cosine_schedule_with_warmup(
    optimizer, num_warmup_steps=len(train_dataset)/BATCH_SIZE*5, num_training_steps=num_train_steps)
loss_fn = torch.nn.CrossEntropyLoss()

### Training loop

In [52]:
train_loss = []
valid_loss = []
train_acc = []
val_acc = []
model = densenet

model.to(device)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

In [ ]:
for epoch in range(NUM_EPOCHS):
    
    tl, ta = training(model, trainloader, optimizer, scheduler, loss_fn, accuracy_score)
    vl, va, conf_mat = validation(model, validloader, loss_fn, accuracy_score, confusion_matrix)
    train_loss.append(tl)
    valid_loss.append(vl)
    train_acc.append(ta)
    val_acc.append(va)
    
    if (epoch+1)%10==0:
        path = 'epoch' + str(epoch) + '.pt'
        torch.save(model.state_dict(), path)
    
    printstr = 'Epoch: '+ str(epoch) + ', Train loss: ' + str(tl) + ', Val loss: ' + str(vl) + ', Train acc: ' + str(ta) + ', Val acc: ' + str(va)
    tqdm.write(printstr)
    


Epoch: 0, Train loss: 0.6531078901428443, Val loss: 0.22686162366442483, Train acc: 0.7699175824175825, Val acc: 0.9342465753424658



Epoch: 1, Train loss: 0.29150327303252377, Val loss: 0.243727308919985, Train acc: 0.9072802197802198, Val acc: 0.9287671232876712



Epoch: 2, Train loss: 0.32601551924433025, Val loss: 0.5667740215993907, Train acc: 0.8873626373626373, Val acc: 0.8



Epoch: 3, Train loss: 0.3515071629004164, Val loss: 0.20610662984521422, Train acc: 0.8921703296703297, Val acc: 0.9342465753424658



Epoch: 4, Train loss: 0.330782912344068, Val loss: 0.24530412542493377, Train acc: 0.8901098901098901, Val acc: 0.9342465753424658


...


Epoch: 11, Train loss: 0.23900089159116641, Val loss: 0.2137270218705478, Train acc: 0.9237637362637363, Val acc: 0.9342465753424658



Epoch: 12, Train loss: 0.26264740665862846, Val loss: 0.2070730591473514, Train acc: 0.915521978021978, Val acc: 0.9452054794520548



Epoch: 13, Train loss: 0.2578445380696884, Val loss: 0.22906624821767416, Train acc: 0.9107142857142857, Val acc: 0.9315068493150684



Epoch: 14, Train loss: 0.22218320633356387, Val loss: 0.18553669207716642, Train acc: 0.9333791208791209, Val acc: 0.9424657534246575



Epoch: 15, Train loss: 0.2141451970725269, Val loss: 0.19514277095664037, Train acc: 0.9320054945054945, Val acc: 0.936986301369863
